<h1 style='color:purple' align='center'>Memprediksi Harga Mobil di Indonesia

Dataset berisikan 1000 data rumah yang dapat diakses melalui
https://www.kaggle.com/code/vincentiusverel/asepbasdat/data?select=train.csv

In [92]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [93]:
df1 = pd.read_csv("data/DATA MOBIL.csv")
df1.head()

,id,perusahaan,nama_mobil,transmisi,odo,tahun,harga
0,504,Bmw,318I,Automatic,140000,2003,98000000
1,1455,Bmw,318I,Automatic,135000,2003,93000000
2,1458,Bmw,318I,Automatic,190000,2003,95000000
3,1528,Bmw,318I,Automatic,175000,2000,72000000
4,410,Bmw,320I,Automatic,30000,2018,495000000


In [94]:
df1 = df1.drop(columns=['id'])
df1.head()

,perusahaan,nama_mobil,transmisi,odo,tahun,harga
0,Bmw,318I,Automatic,140000,2003,98000000
1,Bmw,318I,Automatic,135000,2003,93000000
2,Bmw,318I,Automatic,190000,2003,95000000
3,Bmw,318I,Automatic,175000,2000,72000000
4,Bmw,320I,Automatic,30000,2018,495000000


In [95]:
dummy_perusahaan = pd.get_dummies(df1.perusahaan)
dummy_mobil = pd.get_dummies(df1.nama_mobil)
dummy_transmisi = pd.get_dummies(df1.transmisi)
df1 = df1.drop(columns=['perusahaan', 'nama_mobil', 'transmisi'])

In [96]:
df1.head()

,odo,tahun,harga
0,140000,2003,98000000
1,135000,2003,93000000
2,190000,2003,95000000
3,175000,2000,72000000
4,30000,2018,495000000


In [97]:
train_df = pd.concat([df1, dummy_perusahaan, dummy_mobil, dummy_transmisi], axis='columns')

## Modelling

In [98]:
train_df.head()

,odo,tahun,harga,Bmw,Chevrolet,Daihatsu,Datsun,Ford,Honda,Hyundai,...,Vellfire,Vios,Voxy,Will,X-Trail,Xenia,Xpander,Yaris,Automatic,Manual
0,140000,2003,98000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,135000,2003,93000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,190000,2003,95000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,175000,2000,72000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,30000,2018,495000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [99]:
X = train_df.drop(columns=['harga'])
y = train_df['harga']

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [101]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
svm = SVC(kernel= 'linear', random_state=1, C=0.1)
svm.fit(X_train_std, y_train)
y_svc = svm.predict(X_test_std)
mape = mean_absolute_percentage_error(y_test, y_svc)
print("MAPE SVC:", mape) # MAPE: 0.2598927514929128

MAPE SVC: 0.2506422617593888


In [102]:
def predict_price(perusahaan, mobil, tahun, odo, transmisi ): 
    perusahaan_index = np.where(X.columns==perusahaan.capitalize())[0][0]
    mobil_index = np.where(X.columns==mobil.capitalize())[0][0]
    transmisi_index = np.where(X.columns==transmisi.capitalize())[0][0]
    x = np.zeros(len(X.columns))
    x[0] = odo
    x[1] = tahun
    
    if perusahaan_index >= 0:
        x[perusahaan_index] = 1
        
    if mobil_index >= 0:
        x[mobil_index] = 1
        
    if transmisi_index >= 0:
        x[transmisi_index] = 1
    
    std = sc.transform([x])
    array = svm.predict(std)
    return array[0]

In [103]:
predict_price("toyota","avanza",2010,50000,"manual")

117000000

# Export the tested model to a pickle file

In [104]:
import pickle
with open('../server/models/car_price_prediction.pickle','wb') as f:
    pickle.dump(svm, f)

In [105]:
import json
columns = {
    'data_columns' : [col for col in X.columns]
}

with open('../server/models/data_json/car_columns.json', 'w') as f:
    f.write(json.dumps(columns))